# 核对旅客运输发票进项税数据

## 整理旅客运输报表数据
1.根据发票号进行查重，将**同一发票号码**的进项税抵扣金额进行汇总，比如一张发票出现一正一负两栏数据，则需将金额合计处理。


In [10]:
import pandas as pd
import numpy as np

# 打开excel
invoice_excel_source = '/Users/shihongjin/Downloads/药品事业部旅客运输发票查询报表-20250711-1.xlsx'
invoice_excel_cux_source = '/Users/shihongjin/Downloads/CUX_科目发生额查询报表_110725.xls'

# 检查发票号码列的数据类型和样本数据
invoice_column = '发票号码'  # 请根据实际列名调整
tax_column = '进项税抵扣金额'  # 请根据实际列名调整


# 读取数据时指定发票号码列为文本类型，防止科学计数法
df = pd.read_excel(invoice_excel_source,  dtype={'发票号码': str})

def summarize_invoice_tax_deduction_fixed(df, invoice_column, tax_column):
    """
    汇总同一发票号的进项税抵扣金额（修正版本，处理长数字格式问题）
    """
    # 创建数据副本
    df_clean = df.copy()

    # 确保发票号码列为字符串类型，避免科学计数法
    df_clean[invoice_column] = df_clean[invoice_column].astype(str)

    # 清理数据：移除发票号为空、'nan'、'None'的行
    df_clean = df_clean[~df_clean[invoice_column].isin(['nan', 'None', '', 'NaN'])]
    df_clean = df_clean.dropna(subset=[invoice_column])

    # 处理进项税抵扣金额列：转换为数值类型
    df_clean[tax_column] = pd.to_numeric(df_clean[tax_column], errors='coerce')

    # 移除进项税抵扣金额为空或无效的行
    df_clean = df_clean.dropna(subset=[tax_column])

    print(f"清理后的数据行数: {len(df_clean)}")

    # 查找重复的发票号
    duplicate_invoices = df_clean[df_clean.duplicated(subset=[invoice_column], keep=False)]
    if not duplicate_invoices.empty:
        print(f"发现重复发票号: {len(duplicate_invoices)} 条记录")
        duplicate_invoice_numbers = duplicate_invoices[invoice_column].unique()
        print(f"重复的发票号码数量: {len(duplicate_invoice_numbers)}")
        # 只显示前几个重复的发票号
        print(f"重复的发票号码示例: {list(duplicate_invoice_numbers[:5])}")

    # 按发票号分组汇总
    summary_result = df_clean.groupby(invoice_column).agg({
        tax_column: ['sum', 'count', 'mean']
    }).round(2)

    # 简化列名
    summary_result.columns = ['抵扣金额汇总', '记录数量', '平均金额']
    summary_result = summary_result.reset_index()

    # 添加是否有重复记录的标识
    summary_result['是否重复'] = summary_result['记录数量'] > 1

    # 按汇总金额降序排列
    summary_result = summary_result.sort_values('抵扣金额汇总', ascending=False)

    return summary_result, df_clean


# 执行汇总
try:
    summary_result, cleaned_data = summarize_invoice_tax_deduction_fixed(df, invoice_column, tax_column)

    # 显示汇总结果
    print("\n=== 汇总统计结果 ===")
    print(f"总发票数量: {len(summary_result)}")
    print(f"有重复记录的发票数: {sum(summary_result['是否重复'])}")

    # 显示有重复记录的发票
    duplicates = summary_result[summary_result['是否重复']]
    if not duplicates.empty:
        print(f"\n=== 有重复记录的发票（共{len(duplicates)}张）===")

except Exception as e:
    print(f"处理过程中出现错误: {str(e)}")
    print("请检查列名是否正确")





数据基本信息:
总行数: 862
列名: ['序号', '核算主体名称', '核算主体编码', '发票类型', '发票号码', '发票代码', '发票项目名称', '开票日期', '购方名称', '购方税号', '销方名称', '销方税号', '乘车人/乘机人', '坐席/舱位', '出发日期', '发票金额', '不含税金额', '税额', '核准申报金额', '核准税额', '税率', '进项税抵扣金额', '进项转出金额', '计提销项金额', '计提个税金额', '是否收据', '验真状态', '报账状态', '业务小类', '报账单号', '报账人', '财务初审', '付款状态', '入账状态', '入账期间', '入账时间', '费用凭证号', '凭证摘要']

前5行数据:
   序号          核算主体名称  核算主体编码        发票类型                  发票号码          发票代码  \
0   1  YP00.药品事业部业务实体     222  电子发票(普通发票)  25517000000233447614           NaN   
1   2  YP00.药品事业部业务实体     222  电子发票(普通发票)  25517000000233447614           NaN   
2   3  YP00.药品事业部业务实体     222        客运汽车              17532142  161002119200   
3   4  YP00.药品事业部业务实体     222        客运汽车              11939512  161002319200   
4   5  YP00.药品事业部业务实体     222        客运汽车              10878478  161002119200   

                    发票项目名称        开票日期          购方名称                购方税号  ...  \
0  *运输服务*客运服务费,*运输服务*客运服务费  2025-07-01  云南白药集团股份有限公司  9153000021652214XX  ...   
1

In [15]:
# 保存结果到新的Excel文件（修正版本）
output_file = '/Users/shihongjin/Downloads/旅客运输发票进项税汇总结果.xlsx'

def save_to_excel_with_text_format(summary_result, cleaned_data, duplicates, output_file, invoice_column):
    """
    保存到Excel，确保发票号码列为文本格式
    """
    try:
        # 使用xlsxwriter引擎，可以更好地控制格式
        with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
            # 获取workbook和worksheet对象
            workbook = writer.book

            # 定义文本格式
            text_format = workbook.add_format({'num_format': '@'})  # '@'表示文本格式

            # 保存汇总结果
            summary_result.to_excel(writer, sheet_name='进项税汇总统计', index=False)
            worksheet1 = writer.sheets['进项税汇总统计']

            # 找到发票号码列的位置
            invoice_col_index = summary_result.columns.get_loc(invoice_column)

            # 设置发票号码列为文本格式
            worksheet1.set_column(invoice_col_index, invoice_col_index, None, text_format)

            # 保存清理后的原始数据
            cleaned_data.to_excel(writer, sheet_name='清理后原始数据', index=False)
            worksheet2 = writer.sheets['清理后原始数据']

            # 找到原始数据中发票号码列的位置
            invoice_col_index_orig = list(cleaned_data.columns).index(invoice_column)
            worksheet2.set_column(invoice_col_index_orig, invoice_col_index_orig, None, text_format)

            # 如果有重复记录，单独保存
            if not duplicates.empty:
                duplicate_details = cleaned_data[cleaned_data[invoice_column].isin(duplicates[invoice_column])]
                duplicate_details = duplicate_details.sort_values([invoice_column, tax_column])
                duplicate_details.to_excel(writer, sheet_name='重复发票详情', index=False)
                worksheet3 = writer.sheets['重复发票详情']
                worksheet3.set_column(invoice_col_index_orig, invoice_col_index_orig, None, text_format)

        print(f"\n结果已保存到: {output_file}")
        print("包含以下sheet页:")
        print("- 进项税汇总统计: 按发票号汇总的结果")
        print("- 清理后原始数据: 清理后的原始数据")
        if not duplicates.empty:
            print("- 重复发票详情: 有重复记录的发票详细信息")
        print("\n注意: 发票号码列已设置为文本格式，避免科学计数法显示")

    except Exception as e:
        print(f"保存文件时出错: {str(e)}")
        print("尝试使用openpyxl引擎保存...")

        # 如果xlsxwriter不可用，回退到openpyxl
        try:
            with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
                summary_result.to_excel(writer, sheet_name='进项税汇总统计', index=False)
                cleaned_data.to_excel(writer, sheet_name='清理后原始数据', index=False)
                if not duplicates.empty:
                    duplicate_details = cleaned_data[cleaned_data[invoice_column].isin(duplicates[invoice_column])]
                    duplicate_details = duplicate_details.sort_values([invoice_column, tax_column])
                    duplicate_details.to_excel(writer, sheet_name='重复发票详情', index=False)
            print(f"已使用openpyxl引擎保存到: {output_file}")
            print("注意: 请在Excel中手动设置发票号码列为文本格式")
        except Exception as e2:
            print(f"使用openpyxl保存也失败: {str(e2)}")

# 执行保存
try:
    save_to_excel_with_text_format(summary_result, cleaned_data, duplicates, output_file, invoice_column)
except:
    print("保存功能跳过，请先确保数据处理成功")


保存文件时出错: No module named 'xlsxwriter'
尝试使用openpyxl引擎保存...
已使用openpyxl引擎保存到: /Users/shihongjin/Downloads/旅客运输发票进项税汇总结果.xlsx
注意: 请在Excel中手动设置发票号码列为文本格式
